In [38]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_e161d40bff6e4cd0bcddc04f6bf63856 = 'https://s3.eu.cloud-object-storage.appdomain.cloud'
else:
    endpoint_e161d40bff6e4cd0bcddc04f6bf63856 = 'https://s3.private.eu.cloud-object-storage.appdomain.cloud'

client_e161d40bff6e4cd0bcddc04f6bf63856 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_e161d40bff6e4cd0bcddc04f6bf63856)

body = client_e161d40bff6e4cd0bcddc04f6bf63856.get_object(Bucket='',Key='deliveries.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

In [39]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='', project_access_token='')
pc = project.project_context

In [40]:
team=pd.read_csv(body)

In [87]:
team.head(1000)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5,1,Royal Challengers Bangalore,Delhi Daredevils,6,4,Mandeep Singh,SR Watson,PJ Cummins,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
996,5,1,Royal Challengers Bangalore,Delhi Daredevils,6,5,Mandeep Singh,SR Watson,PJ Cummins,0,...,0,0,0,0,0,0,0,Mandeep Singh,bowled,NaN
997,5,1,Royal Challengers Bangalore,Delhi Daredevils,6,6,KM Jadhav,SR Watson,PJ Cummins,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
998,5,1,Royal Challengers Bangalore,Delhi Daredevils,7,1,SR Watson,KM Jadhav,S Nadeem,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN


# 1) Power Play Overall Stats

In [4]:
def powerplay(teamName):
    runs=0;
    match=0;
    matchid=0;
    for i in range(0,len(team)):
        if(team['batting_team'][i] ==teamName):
            if(team['over'][i]<=6 and team['inning'][i]==1):
                runs+=team['total_runs'][i];
            if(matchid!=team['match_id'][i]):
                matchid=team['match_id'][i];
                match+=1;
        if(team['bowling_team'][i] ==teamName):
            if(team['over'][i]<=6 and team['inning'][i]==2):
                runs+=team['total_runs'][i];
            if(matchid!=team['match_id'][i]):
                matchid=team['match_id'][i];
                match+=1;
    Average=runs/match
    powerplay=Average/6
    return [teamName,runs,match,Average,powerplay]

In [6]:
powerplay_csk=powerplay("Chennai Super Kings")
powerplay_dd=powerplay("Delhi Daredevils")
powerplay_kkr=powerplay("Kolkata Knight Riders")
powerplay_kxip=powerplay("Kings XI Punjab")
powerplay_mi=powerplay("Mumbai Indians")
powerplay_rcb=powerplay("Royal Challengers Bangalore")
powerplay_rr=powerplay("Rajasthan Royals")
powerplay_srh=powerplay("Sunrisers Hyderabad")

powerplay_teams=[powerplay_csk,powerplay_dd,powerplay_kkr,powerplay_kxip,powerplay_mi,powerplay_rcb,powerplay_rr,powerplay_srh]

# print(powerplay_teams)

powerplay_stats=pd.DataFrame(powerplay_teams,columns=['TeamName', 'RunsInPP','MatchesPlayed','AvgPPScore','RunRateInPP'])

powerplay_stats.head(8)

#project.save_data(file_name='powerplay_stats.csv',data=powerplay_stats.to_csv())


,TeamName,RunsInPP,MatchesPlayed,AvgPPScore,RunRateInPP
0,Chennai Super Kings,8255,164,50.335366,8.389228
1,Delhi Daredevils,6717,161,41.720497,6.953416
2,Kolkata Knight Riders,7745,178,43.511236,7.251873
3,Kings XI Punjab,8565,176,48.664773,8.110795
4,Mumbai Indians,9096,187,48.641711,8.106952
5,Royal Challengers Bangalore,7651,180,42.505556,7.084259
6,Rajasthan Royals,6291,147,42.795918,7.132653
7,Sunrisers Hyderabad,5841,108,54.083333,9.013889


In [41]:
# Team indices for usage in further calculations:
def teamIndex(teamName):
    if teamName=="Chennai Super Kings":
        return 0
    elif teamName=="Delhi Daredevils":
        return 1
    elif teamName=="Kolkata Knight Riders":
        return 2
    elif teamName=="Kings XI Punjab":
        return 3
    elif teamName=="Mumbai Indians":
        return 4
    elif teamName=="Royal Challengers Bangalore":
        return 5
    elif teamName=="Rajasthan Royals":
        return 6
    elif teamName=="Sunrisers Hyderabad":
        return 7

# 2) No and Avg No of a Team's batsmen Dismissals during PP, top 10 batsmen dismissed during PP

In [83]:
#To find the maximum number of batsmens dissmissed in powerplay
def noAndAvgOfDismissalsInPP(teamName):
    out=[];
    p=0;
    index=teamIndex(teamName)
    for i in range(0,len(team)):
        if(team['batting_team'][i]==teamName and team['over'][i]<=6):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['player_dismissed'][i]);
    
    return [teamName,len(out),round(len(out)/powerplay_stats['MatchesPlayed'].values[index],0)]

noAndAvgOfDismissalsInPP_csk=noAndAvgOfDismissalsInPP("Chennai Super Kings")
noAndAvgOfDismissalsInPP_dd=noAndAvgOfDismissalsInPP("Delhi Daredevils")
noAndAvgOfDismissalsInPP_kkr=noAndAvgOfDismissalsInPP("Kolkata Knight Riders")
noAndAvgOfDismissalsInPP_kxip=noAndAvgOfDismissalsInPP("Kings XI Punjab")
noAndAvgOfDismissalsInPP_mi=noAndAvgOfDismissalsInPP("Mumbai Indians")
noAndAvgOfDismissalsInPP_rcb=noAndAvgOfDismissalsInPP("Royal Challengers Bangalore")
noAndAvgOfDismissalsInPP_rr=noAndAvgOfDismissalsInPP("Rajasthan Royals")
noAndAvgOfDismissalsInPP_srh=noAndAvgOfDismissalsInPP("Sunrisers Hyderabad")

noAndAvgOfDismissalsInPP_teams=[noAndAvgOfDismissalsInPP_csk,noAndAvgOfDismissalsInPP_dd,noAndAvgOfDismissalsInPP_kkr,noAndAvgOfDismissalsInPP_kxip,noAndAvgOfDismissalsInPP_mi,noAndAvgOfDismissalsInPP_rcb,noAndAvgOfDismissalsInPP_rr,noAndAvgOfDismissalsInPP_srh]

# print(noAndAvgOfDismissals_teams)

noAndAvgOfDismissalsInPP_stats=pd.DataFrame(noAndAvgOfDismissalsInPP_teams,columns=['TeamName', 'NoOfDissimisalsInPP','AvgNoOfDissimisalsInPP'])

noAndAvgOfDismissalsInPP_stats.head(8)

project.save_data(file_name='noAndAvgOfDismissalsInPP_stats.csv',data=noAndAvgOfDismissalsInPP_stats.to_csv())


{'file_name': 'noAndAvgOfDismissalsInPP_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '77eee8be-215d-4dbe-97eb-25db51a93966'}

In [7]:
def playersDismissedInPP(teamName):
    out=[];
    p=0;
    for i in range(0,len(team)):
        if(team['batting_team'][i]==teamName and team['over'][i]<=6):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['player_dismissed'][i]);
    player=[]
    count=[]
    for i in range(0,len(out)):
        if out[i] not in player:
            player.append(out[i]);
            count.append(0);
        else:
            n=player.index(out[i])
            count[n]+=1;  
    for i in range(len(player)):
            for j in range(0, len(player)-i-1):
                if (count[j] < count[j + 1]) :
                    count[j], count[j + 1] = count[j + 1], count[j]
                    player[j], player[j + 1] = player[j + 1], player[j]
    
    return [player[0:10],count[0:10]]

In [10]:
playersDismissedInPP_csk=playersDismissedInPP("Chennai Super Kings")
playersDismissedInPP_dd=playersDismissedInPP("Delhi Daredevils")
playersDismissedInPP_kkr=playersDismissedInPP("Kolkata Knight Riders")
playersDismissedInPP_kxip=playersDismissedInPP("Kings XI Punjab")
playersDismissedInPP_mi=playersDismissedInPP("Mumbai Indians")
playersDismissedInPP_rcb=playersDismissedInPP("Royal Challengers Bangalore")
playersDismissedInPP_rr=playersDismissedInPP("Rajasthan Royals")
playersDismissedInPP_srh=playersDismissedInPP("Sunrisers Hyderabad")

playersDismissedInPP_stats=pd.DataFrame(columns=["TeamName","Player","NoOfTimesDismissed"])

for i in range(0,10):
    row=[["Chennai Super Kings",playersDismissedInPP_csk[0][i],playersDismissedInPP_csk[1][i]]]
    
    playersDismissedInPP_stats = playersDismissedInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Delhi Daredevils",playersDismissedInPP_dd[0][i],playersDismissedInPP_dd[1][i]]]
    
    playersDismissedInPP_stats = playersDismissedInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kolkata Knight Riders",playersDismissedInPP_kkr[0][i],playersDismissedInPP_kkr[1][i]]]
    
    playersDismissedInPP_stats = playersDismissedInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kings XI Punjab",playersDismissedInPP_kxip[0][i],playersDismissedInPP_kxip[1][i]]]
    
    playersDismissedInPP_stats = playersDismissedInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Mumbai Indians",playersDismissedInPP_mi[0][i],playersDismissedInPP_mi[1][i]]]
    
    playersDismissedInPP_stats = playersDismissedInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Rajasthan Royals",playersDismissedInPP_rr[0][i],playersDismissedInPP_rr[1][i]]]
    
    playersDismissedInPP_stats = playersDismissedInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Sunrisers Hyderabad",playersDismissedInPP_srh[0][i],playersDismissedInPP_srh[1][i]]]
    
    playersDismissedInPP_stats = playersDismissedInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)

for i in range(0,10):
    row=[["Royal Challengers Bangalore",playersDismissedInPP_rcb[0][i],playersDismissedInPP_rcb[1][i]]]
    
    playersDismissedInPP_stats = playersDismissedInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    

# playersDismissedInPP_stats.tail(8)


project.save_data(file_name='playersDismissedInPP_stats.csv',data=playersDismissedInPP_stats.to_csv())

{'file_name': 'playersDismissedInPP_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': 'c4879571-ca5b-4e6e-98fa-7257c9de7008'}

# 3) No and Avg No of Wickets taken by team bowlers during PP, top 10 team wicket takers during PP

In [82]:
#To find the maximum number of wickets taken by bowlers in powerplay
def noAndAvgOfWicketsTakenInPP(teamName):
    out=[];
    p=0;
    index=teamIndex(teamName)
    for i in range(0,len(team)):
        if(team['bowling_team'][i]==teamName and team['over'][i]<=6):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['bowler'][i]);
    return [teamName,len(out),round(len(out)/powerplay_stats['MatchesPlayed'].values[index],0)]

noAndAvgOfWicketsTakenInPP_csk=noAndAvgOfWicketsTakenInPP("Chennai Super Kings")
noAndAvgOfWicketsTakenInPP_dd=noAndAvgOfWicketsTakenInPP("Delhi Daredevils")
noAndAvgOfWicketsTakenInPP_kkr=noAndAvgOfWicketsTakenInPP("Kolkata Knight Riders")
noAndAvgOfWicketsTakenInPP_kxip=noAndAvgOfWicketsTakenInPP("Kings XI Punjab")
noAndAvgOfWicketsTakenInPP_mi=noAndAvgOfWicketsTakenInPP("Mumbai Indians")
noAndAvgOfWicketsTakenInPP_rcb=noAndAvgOfWicketsTakenInPP("Royal Challengers Bangalore")
noAndAvgOfWicketsTakenInPP_rr=noAndAvgOfWicketsTakenInPP("Rajasthan Royals")
noAndAvgOfWicketsTakenInPP_srh=noAndAvgOfWicketsTakenInPP("Sunrisers Hyderabad")

noAndAvgOfWicketsTakenInPP_teams=[noAndAvgOfWicketsTakenInPP_csk,noAndAvgOfWicketsTakenInPP_dd,noAndAvgOfWicketsTakenInPP_kkr,noAndAvgOfWicketsTakenInPP_kxip,noAndAvgOfWicketsTakenInPP_mi,noAndAvgOfWicketsTakenInPP_rcb,noAndAvgOfWicketsTakenInPP_rr,noAndAvgOfWicketsTakenInPP_srh]

# print(noAndAvgOfWicketsTakenInPP_teams)

noAndAvgOfWicketsTakenInPP_stats=pd.DataFrame(noAndAvgOfWicketsTakenInPP_teams,columns=['TeamName', 'NoOfWicketsTakenInPP','AvgNoOfWicketsTakenInPP'])

noAndAvgOfWicketsTakenInPP_stats.head(8)

project.save_data(file_name='noAndAvgOfWicketsTakenInPP_stats.csv',data=noAndAvgOfWicketsTakenInPP_stats.to_csv())

{'file_name': 'noAndAvgOfWicketsTakenInPP_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '4428cb64-11a3-4b0d-92ff-bb70b9d1384e'}

In [11]:
def wicketsTakenByBowlersInPP(teamName):
    out=[];
    p=0;
    for i in range(0,len(team)):
        if(team['bowling_team'][i]==teamName and team['over'][i]<=6):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['bowler'][i]);
    player=[]
    count=[]
    for i in range(0,len(out)):
        if out[i] not in player:
            player.append(out[i]);
            count.append(0);
        else:
            n=player.index(out[i])
            count[n]+=1;  
    for i in range(len(player)):
            for j in range(0, len(player)-i-1):
                if (count[j] < count[j + 1]) :
                    count[j], count[j + 1] = count[j + 1], count[j]
                    player[j], player[j + 1] = player[j + 1], player[j]


    return [player[0:10],count[0:10]]

In [13]:
wicketsTakenByBowlersInPP_csk=wicketsTakenByBowlersInPP("Chennai Super Kings")
wicketsTakenByBowlersInPP_dd=wicketsTakenByBowlersInPP("Delhi Daredevils")
wicketsTakenByBowlersInPP_kkr=wicketsTakenByBowlersInPP("Kolkata Knight Riders")
wicketsTakenByBowlersInPP_kxip=wicketsTakenByBowlersInPP("Kings XI Punjab")
wicketsTakenByBowlersInPP_mi=wicketsTakenByBowlersInPP("Mumbai Indians")
wicketsTakenByBowlersInPP_rcb=wicketsTakenByBowlersInPP("Royal Challengers Bangalore")
wicketsTakenByBowlersInPP_rr=wicketsTakenByBowlersInPP("Rajasthan Royals")
wicketsTakenByBowlersInPP_srh=wicketsTakenByBowlersInPP("Sunrisers Hyderabad")

wicketsTakenByBowlersInPP_stats=pd.DataFrame(columns=["TeamName","Player","NoOfWicketsTaken"])

for i in range(0,10):
    row=[["Chennai Super Kings",wicketsTakenByBowlersInPP_csk[0][i],wicketsTakenByBowlersInPP_csk[1][i]]]
    
    wicketsTakenByBowlersInPP_stats = wicketsTakenByBowlersInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Delhi Daredevils",wicketsTakenByBowlersInPP_dd[0][i],wicketsTakenByBowlersInPP_dd[1][i]]]
    
    wicketsTakenByBowlersInPP_stats = wicketsTakenByBowlersInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kolkata Knight Riders",wicketsTakenByBowlersInPP_kkr[0][i],wicketsTakenByBowlersInPP_kkr[1][i]]]
    
    wicketsTakenByBowlersInPP_stats = wicketsTakenByBowlersInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kings XI Punjab",wicketsTakenByBowlersInPP_kxip[0][i],wicketsTakenByBowlersInPP_kxip[1][i]]]
    
    wicketsTakenByBowlersInPP_stats = wicketsTakenByBowlersInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Mumbai Indians",wicketsTakenByBowlersInPP_mi[0][i],wicketsTakenByBowlersInPP_mi[1][i]]]
    
    wicketsTakenByBowlersInPP_stats = wicketsTakenByBowlersInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Rajasthan Royals",wicketsTakenByBowlersInPP_rr[0][i],wicketsTakenByBowlersInPP_rr[1][i]]]
    
    wicketsTakenByBowlersInPP_stats = wicketsTakenByBowlersInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Sunrisers Hyderabad",wicketsTakenByBowlersInPP_srh[0][i],wicketsTakenByBowlersInPP_srh[1][i]]]
    
    wicketsTakenByBowlersInPP_stats = wicketsTakenByBowlersInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Royal Challengers Bangalore",wicketsTakenByBowlersInPP_rcb[0][i],wicketsTakenByBowlersInPP_rcb[1][i]]]
    
    wicketsTakenByBowlersInPP_stats = wicketsTakenByBowlersInPP_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
# wicketsTakenByBowlersInPP_stats.tail()

project.save_data(file_name='wicketsTakenByBowlersInPP_stats.csv',data=wicketsTakenByBowlersInPP_stats.to_csv())

{'file_name': 'wicketsTakenByBowlersInPP_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '3dde68a7-8212-4af4-a512-61f4bb5bad49'}

# 4) Middle Over Overall Stats

In [42]:
def middleOver(teamName):
    runs=0;
    match=0;
    matchid=0;
    for i in range(0,len(team)):
        if(team['batting_team'][i] == teamName):
            if(team['over'][i]>=7 and team['over'][i]<=14 and team['inning'][i]==1):
                runs+=team['total_runs'][i];
            if(matchid!=team['match_id'][i]):
                matchid=team['match_id'][i];
                match+=1;
        if(team['bowling_team'][i] == teamName):
            if(team['over'][i]>=7 and team['over'][i]<=14 and team['inning'][i]==2):
                runs+=team['total_runs'][i];
            if(matchid!=team['match_id'][i]):
                matchid=team['match_id'][i];
                match+=1;
    Average=runs/match
    middleover=Average/7
    return [teamName,runs,match,Average,middleover]

In [44]:
middleOver_csk=middleOver("Chennai Super Kings")
middleOver_dd=middleOver("Delhi Daredevils")
middleOver_kkr=middleOver("Kolkata Knight Riders")
middleOver_kxip=middleOver("Kings XI Punjab")
middleOver_mi=middleOver("Mumbai Indians")
middleOver_rcb=middleOver("Royal Challengers Bangalore")
middleOver_rr=middleOver("Rajasthan Royals")
middleOver_srh=middleOver("Sunrisers Hyderabad")

middleOver_teams=[middleOver_csk,middleOver_dd,middleOver_kkr,middleOver_kxip,middleOver_mi,middleOver_rcb,middleOver_rr,middleOver_srh]

# print(powerplay_teams)

middleOver_stats=pd.DataFrame(middleOver_teams,columns=['TeamName', 'RunsInMiddleOvers','MatchesPlayed','AvgMiddleOverScore','RunRateInMiddleOver'])

# middleOver_stats.head(8)
# project.save_data(file_name='middleOver_stats.csv',data=middleOver_stats.to_csv())

# 5) No and Avg No of a Team's batsmen Dismissals during Middle Overs, top 10 batsmen dismissed during middle overs

In [45]:
# No and Avg No of a Team's batsmen Dismissals during PP, top 10 batsmen dismissed during PP
def noAndAvgOfDismissalsInMiddleOver(teamName):
    out=[];
    p=0;
    index=teamIndex(teamName)
    for i in range(0,len(team)):
        if(team['batting_team'][i]==teamName and team['over'][i]>=7 and team['over'][i]<=14):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['player_dismissed'][i]);
    return [teamName,len(out),round(len(out)/middleOver_stats['MatchesPlayed'].values[index],0)]

noAndAvgOfDismissalsInMiddleOver_csk=noAndAvgOfDismissalsInMiddleOver("Chennai Super Kings")
noAndAvgOfDismissalsInMiddleOver_dd=noAndAvgOfDismissalsInMiddleOver("Delhi Daredevils")
noAndAvgOfDismissalsInMiddleOver_kkr=noAndAvgOfDismissalsInMiddleOver("Kolkata Knight Riders")
noAndAvgOfDismissalsInMiddleOver_kxip=noAndAvgOfDismissalsInMiddleOver("Kings XI Punjab")
noAndAvgOfDismissalsInMiddleOver_mi=noAndAvgOfDismissalsInMiddleOver("Mumbai Indians")
noAndAvgOfDismissalsInMiddleOver_rcb=noAndAvgOfDismissalsInMiddleOver("Royal Challengers Bangalore")
noAndAvgOfDismissalsInMiddleOver_rr=noAndAvgOfDismissalsInMiddleOver("Rajasthan Royals")
noAndAvgOfDismissalsInMiddleOver_srh=noAndAvgOfDismissalsInMiddleOver("Sunrisers Hyderabad")

noAndAvgOfDismissalsInMiddleOver_teams=[noAndAvgOfDismissalsInMiddleOver_csk,noAndAvgOfDismissalsInMiddleOver_dd,noAndAvgOfDismissalsInMiddleOver_kkr,noAndAvgOfDismissalsInMiddleOver_kxip,noAndAvgOfDismissalsInMiddleOver_mi,noAndAvgOfDismissalsInMiddleOver_rcb,noAndAvgOfDismissalsInMiddleOver_rr,noAndAvgOfDismissalsInMiddleOver_srh]


noAndAvgOfDismissalsInMiddleOver_stats=pd.DataFrame(noAndAvgOfDismissalsInMiddleOver_teams,columns=['TeamName', 'NoOfDissimisalsInMiddleOvers','AvgNoOfDissimisalsInMiddleOvers'])

# noAndAvgOfDismissalsInMiddleOver_stats.head(8)

project.save_data(file_name='noAndAvgOfDismissalsInMiddleOver_stats.csv',data=noAndAvgOfDismissalsInMiddleOver_stats.to_csv())

{'file_name': 'noAndAvgOfDismissalsInMiddleOver_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '0b049670-0f20-498a-900e-d150db8ea3a8'}

In [15]:
def playersDismissedInMiddleOvers(teamName):
    out=[];
    p=0;
    for i in range(0,len(team)):
        if(team['batting_team'][i]==teamName and team['over'][i]>=7 and team['over'][i]<=14):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['player_dismissed'][i]);
    player=[];
    count=[];
    for i in range(0,len(out)):
        if out[i] not in player:
            player.append(out[i]);
            count.append(0);
        else:
            n=player.index(out[i])
            count[n]+=1;  
    for i in range(len(player)):
            for j in range(0, len(player)-i-1):
                if (count[j] < count[j + 1]) :
                    count[j], count[j + 1] = count[j + 1], count[j]
                    player[j], player[j + 1] = player[j + 1], player[j]
    
    return [player[0:10],count[0:10]]

In [21]:
playersDismissedInMiddleOvers_csk=playersDismissedInMiddleOvers("Chennai Super Kings")
playersDismissedInMiddleOvers_dd=playersDismissedInMiddleOvers("Delhi Daredevils")
playersDismissedInMiddleOvers_kkr=playersDismissedInMiddleOvers("Kolkata Knight Riders")
playersDismissedInMiddleOvers_kxip=playersDismissedInMiddleOvers("Kings XI Punjab")
playersDismissedInMiddleOvers_mi=playersDismissedInMiddleOvers("Mumbai Indians")
playersDismissedInMiddleOvers_rcb=playersDismissedInMiddleOvers("Royal Challengers Bangalore")
playersDismissedInMiddleOvers_rr=playersDismissedInMiddleOvers("Rajasthan Royals")
playersDismissedInMiddleOvers_srh=playersDismissedInMiddleOvers("Sunrisers Hyderabad")

playersDismissedInMiddleOvers_stats=pd.DataFrame(columns=["TeamName","Player","NoOfTimesDismissed"])

for i in range(0,10):
    row=[["Chennai Super Kings",playersDismissedInMiddleOvers_csk[0][i],playersDismissedInMiddleOvers_csk[1][i]]]
    
    playersDismissedInMiddleOvers_stats = playersDismissedInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Delhi Daredevils",playersDismissedInMiddleOvers_dd[0][i],playersDismissedInMiddleOvers_dd[1][i]]]
    
    playersDismissedInMiddleOvers_stats = playersDismissedInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kolkata Knight Riders",playersDismissedInMiddleOvers_kkr[0][i],playersDismissedInMiddleOvers_kkr[1][i]]]
    
    playersDismissedInMiddleOvers_stats = playersDismissedInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kings XI Punjab",playersDismissedInMiddleOvers_kxip[0][i],playersDismissedInMiddleOvers_kxip[1][i]]]
    
    playersDismissedInMiddleOvers_stats = playersDismissedInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Mumbai Indians",playersDismissedInMiddleOvers_mi[0][i],playersDismissedInMiddleOvers_mi[1][i]]]
    
    playersDismissedInMiddleOvers_stats = playersDismissedInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Rajasthan Royals",playersDismissedInMiddleOvers_rr[0][i],playersDismissedInMiddleOvers_rr[1][i]]]
    
    playersDismissedInMiddleOvers_stats = playersDismissedInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Sunrisers Hyderabad",playersDismissedInMiddleOvers_srh[0][i],playersDismissedInMiddleOvers_srh[1][i]]]
    
    playersDismissedInMiddleOvers_stats = playersDismissedInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)

for i in range(0,10):
    row=[["Royal Challengers Bangalore",playersDismissedInMiddleOvers_rcb[0][i],playersDismissedInMiddleOvers_rcb[1][i]]]
    
    playersDismissedInMiddleOvers_stats = playersDismissedInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    

    

    
# playersDismissedInMiddleOvers_stats.tail(20)

project.save_data(file_name='playersDismissedInMiddleOvers_stats.csv',data=playersDismissedInMiddleOvers_stats.to_csv())

{'file_name': 'playersDismissedInMiddleOvers_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '0a33ef0e-dfeb-44fc-a07c-7d162d34690b'}

# 6) No and Avg No of Wickets taken by team bowlers during Middle Overs, top 10 team wicket takers during Middle Overs

In [45]:
#To find the maximum number of wickets taken by bowlers in powerplay
def noAndAvgOfWicketsTakenInMiddleOvers(teamName):
    out=[];
    p=0;
    index=teamIndex(teamName)
    for i in range(0,len(team)):
        if(team['bowling_team'][i]==teamName and team['over'][i]>=7 and team['over'][i]<=14):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['bowler'][i]);
    return [teamName,len(out),round(len(out)/middleOver_stats['MatchesPlayed'].values[index],0)]

noAndAvgOfWicketsTakenInMiddleOvers_csk=noAndAvgOfWicketsTakenInMiddleOvers("Chennai Super Kings")
noAndAvgOfWicketsTakenInMiddleOvers_dd=noAndAvgOfWicketsTakenInMiddleOvers("Delhi Daredevils")
noAndAvgOfWicketsTakenInMiddleOvers_kkr=noAndAvgOfWicketsTakenInMiddleOvers("Kolkata Knight Riders")
noAndAvgOfWicketsTakenInMiddleOvers_kxip=noAndAvgOfWicketsTakenInMiddleOvers("Kings XI Punjab")
noAndAvgOfWicketsTakenInMiddleOvers_mi=noAndAvgOfWicketsTakenInMiddleOvers("Mumbai Indians")
noAndAvgOfWicketsTakenInMiddleOvers_rcb=noAndAvgOfWicketsTakenInMiddleOvers("Royal Challengers Bangalore")
noAndAvgOfWicketsTakenInMiddleOvers_rr=noAndAvgOfWicketsTakenInMiddleOvers("Rajasthan Royals")
noAndAvgOfWicketsTakenInMiddleOvers_srh=noAndAvgOfWicketsTakenInMiddleOvers("Sunrisers Hyderabad")

noAndAvgOfWicketsTakenInMiddleOvers_teams=[noAndAvgOfWicketsTakenInMiddleOvers_csk,noAndAvgOfWicketsTakenInMiddleOvers_dd,noAndAvgOfWicketsTakenInMiddleOvers_kkr,noAndAvgOfWicketsTakenInMiddleOvers_kxip,noAndAvgOfWicketsTakenInMiddleOvers_mi,noAndAvgOfWicketsTakenInMiddleOvers_rcb,noAndAvgOfWicketsTakenInMiddleOvers_rr,noAndAvgOfWicketsTakenInMiddleOvers_srh]

# print(noAndAvgOfWicketsTakenInPP_teams)

noAndAvgOfWicketsTakenInMiddleOvers_stats=pd.DataFrame(noAndAvgOfWicketsTakenInMiddleOvers_teams,columns=['TeamName', 'NoOfWicketsTakenInMiddleOvers','AvgNoOfWicketsTakenInMiddleOvers'])

# noAndAvgOfWicketsTakenInMiddleOvers_stats.head(8)


project.save_data(file_name='noAndAvgOfWicketsTakenInMiddleOvers_stats.csv',data=noAndAvgOfWicketsTakenInMiddleOvers_stats.to_csv())

{'file_name': 'noAndAvgOfWicketsTakenInMiddleOvers_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '17360df7-63ae-4b57-900b-5e562789759f'}

In [22]:
def wicketsTakenByBowlersInMiddleOvers(teamName):
    out=[];
    p=0;
    for i in range(0,len(team)):
        if(team['bowling_team'][i]==teamName and team['over'][i]>=7 and team['over'][i]<=14):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['bowler'][i]);
    player=[];
    count=[];
    for i in range(0,len(out)):
        if out[i] not in player:
            player.append(out[i]);
            count.append(0);
        else:
            n=player.index(out[i])
            count[n]+=1;  
    for i in range(len(player)):
            for j in range(0, len(player)-i-1):
                if (count[j] < count[j + 1]) :
                    count[j], count[j + 1] = count[j + 1], count[j]
                    player[j], player[j + 1] = player[j + 1], player[j]
    
    return [player[0:10],count[0:10]]

In [24]:
wicketsTakenByBowlersInMiddleOvers_csk=wicketsTakenByBowlersInMiddleOvers("Chennai Super Kings")
wicketsTakenByBowlersInMiddleOvers_dd=wicketsTakenByBowlersInMiddleOvers("Delhi Daredevils")
wicketsTakenByBowlersInMiddleOvers_kkr=wicketsTakenByBowlersInMiddleOvers("Kolkata Knight Riders")
wicketsTakenByBowlersInMiddleOvers_kxip=wicketsTakenByBowlersInMiddleOvers("Kings XI Punjab")
wicketsTakenByBowlersInMiddleOvers_mi=wicketsTakenByBowlersInMiddleOvers("Mumbai Indians")
wicketsTakenByBowlersInMiddleOvers_rcb=wicketsTakenByBowlersInMiddleOvers("Royal Challengers Bangalore")
wicketsTakenByBowlersInMiddleOvers_rr=wicketsTakenByBowlersInMiddleOvers("Rajasthan Royals")
wicketsTakenByBowlersInMiddleOvers_srh=wicketsTakenByBowlersInMiddleOvers("Sunrisers Hyderabad")

wicketsTakenByBowlersInMiddleOvers_stats=pd.DataFrame(columns=["TeamName","Player","NoOfWicketsTaken"])

for i in range(0,10):
    row=[["Chennai Super Kings",wicketsTakenByBowlersInMiddleOvers_csk[0][i],wicketsTakenByBowlersInMiddleOvers_csk[1][i]]]
    
    wicketsTakenByBowlersInMiddleOvers_stats = wicketsTakenByBowlersInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Delhi Daredevils",wicketsTakenByBowlersInPP_dd[0][i],wicketsTakenByBowlersInPP_dd[1][i]]]
    
    wicketsTakenByBowlersInMiddleOvers_stats = wicketsTakenByBowlersInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kolkata Knight Riders",wicketsTakenByBowlersInMiddleOvers_kkr[0][i],wicketsTakenByBowlersInMiddleOvers_kkr[1][i]]]
    
    wicketsTakenByBowlersInMiddleOvers_stats = wicketsTakenByBowlersInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kings XI Punjab",wicketsTakenByBowlersInMiddleOvers_kxip[0][i],wicketsTakenByBowlersInMiddleOvers_kxip[1][i]]]
    
    wicketsTakenByBowlersInMiddleOvers_stats = wicketsTakenByBowlersInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Mumbai Indians",wicketsTakenByBowlersInMiddleOvers_mi[0][i],wicketsTakenByBowlersInMiddleOvers_mi[1][i]]]
    
    wicketsTakenByBowlersInMiddleOvers_stats = wicketsTakenByBowlersInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Rajasthan Royals",wicketsTakenByBowlersInMiddleOvers_rr[0][i],wicketsTakenByBowlersInMiddleOvers_rr[1][i]]]
    
    wicketsTakenByBowlersInMiddleOvers_stats = wicketsTakenByBowlersInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Sunrisers Hyderabad",wicketsTakenByBowlersInMiddleOvers_srh[0][i],wicketsTakenByBowlersInMiddleOvers_srh[1][i]]]
    
    wicketsTakenByBowlersInMiddleOvers_stats = wicketsTakenByBowlersInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Royal Challengers Bangalore",wicketsTakenByBowlersInMiddleOvers_rcb[0][i],wicketsTakenByBowlersInMiddleOvers_rcb[1][i]]]
    
    wicketsTakenByBowlersInMiddleOvers_stats = wicketsTakenByBowlersInMiddleOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
 
# wicketsTakenByBowlersInMiddleOvers_stats.tail(10)

project.save_data(file_name='wicketsTakenByBowlersInMiddleOvers_stats.csv',data=wicketsTakenByBowlersInMiddleOvers_stats.to_csv())

{'file_name': 'wicketsTakenByBowlersInMiddleOvers_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': 'adc3e98c-c8a7-4184-a1ec-38f397739c7f'}

# 7) Death Overs Overall Stats

In [25]:
def deathOver(teamName):
    runs=0;
    match=0;
    matchid=0;
    for i in range(0,len(team)):
        if(team['batting_team'][i] == teamName):
            if(team['over'][i]<=15 and team['over'][i]>=20 and team['inning'][i]==1):
                runs+=team['total_runs'][i];
            if(matchid!=team['match_id'][i]):
                matchid=team['match_id'][i];
                match+=1;
        if(team['bowling_team'][i] == teamName):
            if(team['over'][i]<=6 and team['inning'][i]==2):
                runs+=team['total_runs'][i];
            if(matchid!=team['match_id'][i]):
                matchid=team['match_id'][i];
                match+=1;
    Average=runs/match
    deathover=Average/6
    return [teamName,runs,match,Average,deathover]

In [90]:
deathOver_csk=deathOver("Chennai Super Kings")
deathOver_dd=deathOver("Delhi Daredevils")
deathOver_kkr=deathOver("Kolkata Knight Riders")
deathOver_kxip=deathOver("Kings XI Punjab")
deathOver_mi=deathOver("Mumbai Indians")
deathOver_rcb=deathOver("Royal Challengers Bangalore")
deathOver_rr=deathOver("Rajasthan Royals")
deathOver_srh=deathOver("Sunrisers Hyderabad")

deathOver_teams=[deathOver_csk,deathOver_dd,deathOver_kkr,deathOver_kxip,deathOver_mi,deathOver_rcb,deathOver_rr,deathOver_srh]

# print(powerplay_teams)

deathOver_stats=pd.DataFrame(deathOver_teams,columns=['TeamName', 'RunsInDeathOvers','MatchesPlayed','AvgDeathOverScore','RunRateInDeathOver'])

deathOver_stats.head(8)

project.save_data(file_name='deathOver_stats.csv',data=deathOver_stats.to_csv())

{'file_name': 'deathOver_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': 'dcd1cd11-3f70-408a-8ebf-873eb81bbb88'}

# 8) No and Avg No of a Team's batsmen Dismissals during Death Overs, top 10 batsmen dismissed during death overs

In [58]:
def noAndAvgOfDismissalsInDeathOver(teamName):
    out=[];
    p=0;
    index=teamIndex(teamName)
    for i in range(0,len(team)):
        if(team['batting_team'][i]==teamName and team['over'][i]>=15 and team['over'][i]<=20):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['player_dismissed'][i]);
    return [teamName,len(out),round(len(out)/deathOver_stats['MatchesPlayed'].values[index],0)]


noAndAvgOfDismissalsInDeathOver_csk=noAndAvgOfDismissalsInDeathOver("Chennai Super Kings")
noAndAvgOfDismissalsInDeathOver_dd=noAndAvgOfDismissalsInDeathOver("Delhi Daredevils")
noAndAvgOfDismissalsInDeathOver_kkr=noAndAvgOfDismissalsInDeathOver("Kolkata Knight Riders")
noAndAvgOfDismissalsInDeathOver_kxip=noAndAvgOfDismissalsInDeathOver("Kings XI Punjab")
noAndAvgOfDismissalsInDeathOver_mi=noAndAvgOfDismissalsInDeathOver("Mumbai Indians")
noAndAvgOfDismissalsInDeathOver_rcb=noAndAvgOfDismissalsInDeathOver("Royal Challengers Bangalore")
noAndAvgOfDismissalsInDeathOver_rr=noAndAvgOfDismissalsInDeathOver("Rajasthan Royals")
noAndAvgOfDismissalsInDeathOver_srh=noAndAvgOfDismissalsInDeathOver("Sunrisers Hyderabad")

noAndAvgOfDismissalsInDeathOver_teams=[noAndAvgOfDismissalsInDeathOver_csk,noAndAvgOfDismissalsInDeathOver_dd,noAndAvgOfDismissalsInDeathOver_kkr,noAndAvgOfDismissalsInDeathOver_kxip,noAndAvgOfDismissalsInMiddleOver_mi,noAndAvgOfDismissalsInDeathOver_rcb,noAndAvgOfDismissalsInDeathOver_rr,noAndAvgOfDismissalsInDeathOver_srh]


noAndAvgOfDismissalsInDeathOver_stats=pd.DataFrame(noAndAvgOfDismissalsInDeathOver_teams,columns=['TeamName', 'NoOfDissimisalsInDeathOvers','AvgNoOfDissimisalsInDeathOvers'])

noAndAvgOfDismissalsInMiddleOver_stats.head(8)

# project.save_data(file_name='noAndAvgOfDismissalsInDeathOver_stats.csv',data=noAndAvgOfDismissalsInDeathOver_stats.to_csv())

,TeamName,NoOfDissimisalsInMiddleOvers,AvgNoOfDissimisalsInMiddleOvers
0,Chennai Super Kings,287,2.0
1,Delhi Daredevils,312,2.0
2,Kolkata Knight Riders,316,2.0
3,Kings XI Punjab,359,2.0
4,Mumbai Indians,371,2.0
5,Royal Challengers Bangalore,343,2.0
6,Rajasthan Royals,318,2.0
7,Sunrisers Hyderabad,182,2.0


In [26]:
def playersDismissedInDeathOvers(teamName):
    out=[];
    p=0;
    for i in range(0,len(team)):
        if(team['batting_team'][i]==teamName and team['over'][i]>=15 and team['over'][i]<=20):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['player_dismissed'][i]);
    player=[];
    count=[];
    for i in range(0,len(out)):
        if out[i] not in player:
            player.append(out[i]);
            count.append(0);
        else:
            n=player.index(out[i])
            count[n]+=1;  
    for i in range(len(player)):
            for j in range(0, len(player)-i-1):
                if (count[j] < count[j + 1]) :
                    count[j], count[j + 1] = count[j + 1], count[j]
                    player[j], player[j + 1] = player[j + 1], player[j]
    return [player[0:10],count[0:10]]

In [28]:
playersDismissedInDeathOvers_csk=playersDismissedInDeathOvers("Chennai Super Kings")
playersDismissedInDeathOvers_dd=playersDismissedInDeathOvers("Delhi Daredevils")
playersDismissedInDeathOvers_kkr=playersDismissedInDeathOvers("Kolkata Knight Riders")
playersDismissedInDeathOvers_kxip=playersDismissedInDeathOvers("Kings XI Punjab")
playersDismissedInDeathOvers_mi=playersDismissedInDeathOvers("Mumbai Indians")
playersDismissedInDeathOvers_rcb=playersDismissedInDeathOvers("Royal Challengers Bangalore")
playersDismissedInDeathOvers_rr=playersDismissedInDeathOvers("Rajasthan Royals")
playersDismissedInDeathOvers_srh=playersDismissedInDeathOvers("Sunrisers Hyderabad")

playersDismissedInDeathOvers_stats=pd.DataFrame(columns=["TeamName","Player","NoOfTimesDismissed"])

for i in range(0,10):
    row=[["Chennai Super Kings",playersDismissedInDeathOvers_csk[0][i],playersDismissedInDeathOvers_csk[1][i]]]
    
    playersDismissedInDeathOvers_stats = playersDismissedInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Delhi Daredevils",playersDismissedInDeathOvers_dd[0][i],playersDismissedInDeathOvers_dd[1][i]]]
    
    playersDismissedInDeathOvers_stats = playersDismissedInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kolkata Knight Riders",playersDismissedInDeathOvers_kkr[0][i],playersDismissedInDeathOvers_kkr[1][i]]]
    
    playersDismissedInDeathOvers_stats = playersDismissedInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kings XI Punjab",playersDismissedInDeathOvers_kxip[0][i],playersDismissedInDeathOvers_kxip[1][i]]]
    
    playersDismissedInDeathOvers_stats = playersDismissedInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Mumbai Indians",playersDismissedInDeathOvers_mi[0][i],playersDismissedInDeathOvers_mi[1][i]]]
    
    playersDismissedInDeathOvers_stats = playersDismissedInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Rajasthan Royals",playersDismissedInDeathOvers_rr[0][i],playersDismissedInDeathOvers_rr[1][i]]]
    
    playersDismissedInDeathOvers_stats = playersDismissedInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Sunrisers Hyderabad",playersDismissedInDeathOvers_srh[0][i],playersDismissedInDeathOvers_srh[1][i]]]
    
    playersDismissedInDeathOvers_stats = playersDismissedInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)

for i in range(0,10):
    row=[["Royal Challengers Bangalore",playersDismissedInDeathOvers_rcb[0][i],playersDismissedInDeathOvers_rcb[1][i]]]
    
    playersDismissedInDeathOvers_stats = playersDismissedInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfTimesDismissed"]),ignore_index=True)

# playersDismissedInDeathOvers_stats.tail(20)

project.save_data(file_name='playersDismissedInDeathOvers_stats.csv',data=playersDismissedInDeathOvers_stats.to_csv())

{'file_name': 'playersDismissedInDeathOvers_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': 'c03df3ac-81f8-40a1-b8b3-b91bdfbeae29'}

# 9) No and Avg No of Wickets taken by team bowlers during Death Overs, top 10 team wicket takers during death Overs

In [62]:
def noAndAvgOfWicketsTakenInDeathOvers(teamName):
    out=[];
    p=0;
    index=teamIndex(teamName)
    for i in range(0,len(team)):
        if(team['bowling_team'][i]==teamName and team['over'][i]>=15 and team['over'][i]<=20):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['bowler'][i]);
    return [teamName,len(out),round(len(out)/middleOver_stats['MatchesPlayed'].values[index],0)]

noAndAvgOfWicketsTakenInDeathOvers_csk=noAndAvgOfWicketsTakenInDeathOvers("Chennai Super Kings")
noAndAvgOfWicketsTakenInDeathOvers_dd=noAndAvgOfWicketsTakenInDeathOvers("Delhi Daredevils")
noAndAvgOfWicketsTakenInDeathOvers_kkr=noAndAvgOfWicketsTakenInDeathOvers("Kolkata Knight Riders")
noAndAvgOfWicketsTakenInDeathOvers_kxip=noAndAvgOfWicketsTakenInDeathOvers("Kings XI Punjab")
noAndAvgOfWicketsTakenInDeathOvers_mi=noAndAvgOfWicketsTakenInDeathOvers("Mumbai Indians")
noAndAvgOfWicketsTakenInDeathOvers_rcb=noAndAvgOfWicketsTakenInDeathOvers("Royal Challengers Bangalore")
noAndAvgOfWicketsTakenInDeathOvers_rr=noAndAvgOfWicketsTakenInDeathOvers("Rajasthan Royals")
noAndAvgOfWicketsTakenInDeathOvers_srh=noAndAvgOfWicketsTakenInDeathOvers("Sunrisers Hyderabad")

noAndAvgOfWicketsTakenInDeathOvers_teams=[noAndAvgOfWicketsTakenInDeathOvers_csk,noAndAvgOfWicketsTakenInDeathOvers_dd,noAndAvgOfWicketsTakenInDeathOvers_kkr,noAndAvgOfWicketsTakenInDeathOvers_kxip,noAndAvgOfWicketsTakenInDeathOvers_mi,noAndAvgOfWicketsTakenInDeathOvers_rcb,noAndAvgOfWicketsTakenInDeathOvers_rr,noAndAvgOfWicketsTakenInDeathOvers_srh]


noAndAvgOfWicketsTakenInDeathOvers_stats=pd.DataFrame(noAndAvgOfWicketsTakenInDeathOvers_teams,columns=['TeamName', 'NoOfWicketsTakenInDeathOvers','AvgNoOfWicketsTakenInDeathOvers'])

# noAndAvgOfWicketsTakenInDeathOvers_stats.head(8)


project.save_data(file_name='noAndAvgOfWicketsTakenInDeathOvers_stats.csv',data=noAndAvgOfWicketsTakenInDeathOvers_stats.to_csv())

{'file_name': 'noAndAvgOfWicketsTakenInDeathOvers_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '53eaa125-a2d0-4f17-94ea-580f116ff8a7'}

In [30]:
def wicketsTakenByBowlersInDeathOvers(teamName):
    out=[];
    p=0;
    for i in range(0,len(team)):
        if(team['bowling_team'][i]==teamName and team['over'][i]>=15 and team['over'][i]<=20):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['bowler'][i]);
    player=[];
    count=[];
    for i in range(0,len(out)):
        if out[i] not in player:
            player.append(out[i]);
            count.append(0);
        else:
            n=player.index(out[i])
            count[n]+=1;  
    for i in range(len(player)):
            for j in range(0, len(player)-i-1):
                if (count[j] < count[j + 1]) :
                    count[j], count[j + 1] = count[j + 1], count[j]
                    player[j], player[j + 1] = player[j + 1], player[j]
                    
    return [player[0:10],count[0:10]]

In [32]:
wicketsTakenByBowlersInDeathOvers_csk=wicketsTakenByBowlersInDeathOvers("Chennai Super Kings")
wicketsTakenByBowlersInDeathOvers_dd=wicketsTakenByBowlersInDeathOvers("Delhi Daredevils")
wicketsTakenByBowlersInDeathOvers_kkr=wicketsTakenByBowlersInDeathOvers("Kolkata Knight Riders")
wicketsTakenByBowlersInDeathOvers_kxip=wicketsTakenByBowlersInDeathOvers("Kings XI Punjab")
wicketsTakenByBowlersInDeathOvers_mi=wicketsTakenByBowlersInDeathOvers("Mumbai Indians")
wicketsTakenByBowlersInDeathOvers_rcb=wicketsTakenByBowlersInDeathOvers("Royal Challengers Bangalore")
wicketsTakenByBowlersInDeathOvers_rr=wicketsTakenByBowlersInDeathOvers("Rajasthan Royals")
wicketsTakenByBowlersInDeathOvers_srh=wicketsTakenByBowlersInDeathOvers("Sunrisers Hyderabad")

wicketsTakenByBowlersInDeathOvers_stats=pd.DataFrame(columns=["TeamName","Player","NoOfWicketsTaken"])

for i in range(0,10):
    row=[["Chennai Super Kings",wicketsTakenByBowlersInDeathOvers_csk[0][i],wicketsTakenByBowlersInDeathOvers_csk[1][i]]]
    
    wicketsTakenByBowlersInDeathOvers_stats = wicketsTakenByBowlersInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Delhi Daredevils",wicketsTakenByBowlersInDeathOvers_dd[0][i],wicketsTakenByBowlersInDeathOvers_dd[1][i]]]
    
    wicketsTakenByBowlersInDeathOvers_stats = wicketsTakenByBowlersInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kolkata Knight Riders",wicketsTakenByBowlersInDeathOvers_kkr[0][i],wicketsTakenByBowlersInDeathOvers_kkr[1][i]]]
    
    wicketsTakenByBowlersInDeathOvers_stats = wicketsTakenByBowlersInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kings XI Punjab",wicketsTakenByBowlersInDeathOvers_kxip[0][i],wicketsTakenByBowlersInDeathOvers_kxip[1][i]]]
    
    wicketsTakenByBowlersInDeathOvers_stats = wicketsTakenByBowlersInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Mumbai Indians",wicketsTakenByBowlersInDeathOvers_mi[0][i],wicketsTakenByBowlersInDeathOvers_mi[1][i]]]
    
    wicketsTakenByBowlersInDeathOvers_stats = wicketsTakenByBowlersInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Rajasthan Royals",wicketsTakenByBowlersInDeathOvers_rr[0][i],wicketsTakenByBowlersInDeathOvers_rr[1][i]]]
    
    wicketsTakenByBowlersInDeathOvers_stats = wicketsTakenByBowlersInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Sunrisers Hyderabad",wicketsTakenByBowlersInDeathOvers_srh[0][i],wicketsTakenByBowlersInDeathOvers_srh[1][i]]]
    
    wicketsTakenByBowlersInDeathOvers_stats = wicketsTakenByBowlersInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)

for i in range(0,10):
    row=[["Royal Challengers Bangalore",wicketsTakenByBowlersInDeathOvers_rcb[0][i],wicketsTakenByBowlersInDeathOvers_rcb[1][i]]]
    
    wicketsTakenByBowlersInDeathOvers_stats = wicketsTakenByBowlersInDeathOvers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","NoOfWicketsTaken"]),ignore_index=True)
# wicketsTakenByBowlersInDeathOvers_stats.tail(10)

project.save_data(file_name='wicketsTakenByBowlersInDeathOvers_stats.csv',data=wicketsTakenByBowlersInDeathOvers_stats.to_csv())

{'file_name': 'wicketsTakenByBowlersInDeathOvers_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '9ad9fc90-f296-48b5-8c3e-8f745df24efa'}

# Comparison between diferent powerplays [NOT GOING TO INCLUDE THIS]

In [ ]:
if(powerplay<middleover and powerplay<Deathover):
    print("Powerplay is an issue of concern and the management needs to concentrate")
elif(middleover<powerplay and middleover<Deathover):
    print("Middleover is an issue of concern and the management needs to concentrate")
elif(Deathover<powerplay and Deathover<middleover):
    print("Death over is an issue of concern and the management needs to concentrate")

Death over is an issue of concern and the management needs to concentrate


# 10) Top 10 Highest Run Scorers of each team

In [33]:
def highestRunScorers(teamName):
    batsmen=[];
    runs=[];
    for i in range(0,len(team)):
        if(team['batting_team'][i]==teamName):
            if(team['batsman'][i] not in batsmen):
                batsmen.append(team['batsman'][i])
                runs.append(team['batsman_runs'][i])
            else:
                p=batsmen.index(team['batsman'][i])
                runs[p]+=team['batsman_runs'][i]
    for i in range(len(batsmen)):
            for j in range(0, len(batsmen)-i-1):
                if (runs[j] < runs[j + 1]) :
                    runs[j], runs[j + 1] = runs[j + 1], runs[j]
                    batsmen[j], batsmen[j + 1] = batsmen[j + 1], batsmen[j]
                    
    return [batsmen[0:10],runs[0:10]]

In [35]:
highestRunScorers_csk=highestRunScorers("Chennai Super Kings")
highestRunScorers_dd=highestRunScorers("Delhi Daredevils")
highestRunScorers_kkr=highestRunScorers("Kolkata Knight Riders")
highestRunScorers_kxip=highestRunScorers("Kings XI Punjab")
highestRunScorers_mi=highestRunScorers("Mumbai Indians")
highestRunScorers_rcb=highestRunScorers("Royal Challengers Bangalore")
highestRunScorers_rr=highestRunScorers("Rajasthan Royals")
highestRunScorers_srh=highestRunScorers("Sunrisers Hyderabad")

highestRunScorers_stats=pd.DataFrame(columns=["TeamName","Player","RunsScored"])

for i in range(0,10):
    row=[["Chennai Super Kings",highestRunScorers_csk[0][i],highestRunScorers_csk[1][i]]]
    
    highestRunScorers_stats = highestRunScorers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","RunsScored"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Delhi Daredevils",highestRunScorers_dd[0][i],highestRunScorers_dd[1][i]]]
    
    highestRunScorers_stats = highestRunScorers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","RunsScored"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kolkata Knight Riders",highestRunScorers_kkr[0][i],highestRunScorers_kkr[1][i]]]
    
    highestRunScorers_stats = highestRunScorers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","RunsScored"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kings XI Punjab",highestRunScorers_kxip[0][i],highestRunScorers_kxip[1][i]]]
    
    highestRunScorers_stats = highestRunScorers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","RunsScored"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Mumbai Indians",highestRunScorers_mi[0][i],highestRunScorers_mi[1][i]]]
    
    highestRunScorers_stats = highestRunScorers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","RunsScored"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Rajasthan Royals",highestRunScorers_rr[0][i],highestRunScorers_rr[1][i]]]
    
    highestRunScorers_stats = highestRunScorers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","RunsScored"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Sunrisers Hyderabad",highestRunScorers_srh[0][i],highestRunScorers_srh[1][i]]]
    
    highestRunScorers_stats = highestRunScorers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","RunsScored"]),ignore_index=True)

for i in range(0,10):
    row=[["Royal Challengers Bangalore",highestRunScorers_rcb[0][i],highestRunScorers_rcb[1][i]]]
    
    highestRunScorers_stats = highestRunScorers_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","RunsScored"]),ignore_index=True)
    
# highestRunScorers_stats.tail(20)

project.save_data(file_name='highestRunScorers_stats.csv',data=highestRunScorers_stats.to_csv())

{'file_name': 'highestRunScorers_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': '60da261c-7d71-488d-ae30-994687cd3bf8'}

# 11) Top 10 wicket takers of each team

In [36]:
def highestWicketTaker(teamName):
    out=[];
    p=0;
    for i in range(0,len(team)):
        if(team['bowling_team'][i]==teamName):
            if(pd.isnull(team['player_dismissed'][i])):
                p+=1;
            else:
                out.append(team['bowler'][i]);
    player=[];
    count=[];
    for i in range(0,len(out)):
        if out[i] not in player:
            player.append(out[i]);
            count.append(0);
        else:
            n=player.index(out[i])
            count[n]+=1;  
    for i in range(len(player)):
            for j in range(0, len(player)-i-1):
                if (count[j] < count[j + 1]) :
                    count[j], count[j + 1] = count[j + 1], count[j]
                    player[j], player[j + 1] = player[j + 1], player[j]
                    
    return [player[0:10],count[0:10]]

In [37]:
highestWicketTaker_csk=highestWicketTaker("Chennai Super Kings")
highestWicketTaker_dd=highestWicketTaker("Delhi Daredevils")
highestWicketTaker_kkr=highestWicketTaker("Kolkata Knight Riders")
highestWicketTaker_kxip=highestWicketTaker("Kings XI Punjab")
highestWicketTaker_mi=highestWicketTaker("Mumbai Indians")
highestWicketTaker_rcb=highestWicketTaker("Royal Challengers Bangalore")
highestWicketTaker_rr=highestWicketTaker("Rajasthan Royals")
highestWicketTaker_srh=highestWicketTaker("Sunrisers Hyderabad")

highestWicketTaker_stats=pd.DataFrame(columns=["TeamName","Player","WicketsTaken"])

for i in range(0,10):
    row=[["Chennai Super Kings",highestWicketTaker_csk[0][i],highestWicketTaker_csk[1][i]]]
    
    highestWicketTaker_stats = highestWicketTaker_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","WicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Delhi Daredevils",highestWicketTaker_dd[0][i],highestWicketTaker_dd[1][i]]]
    
    highestWicketTaker_stats = highestWicketTaker_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","WicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kolkata Knight Riders",highestWicketTaker_kkr[0][i],highestWicketTaker_kkr[1][i]]]
    
    highestWicketTaker_stats = highestWicketTaker_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","WicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Kings XI Punjab",highestWicketTaker_kxip[0][i],highestWicketTaker_kxip[1][i]]]
    
    highestWicketTaker_stats = highestWicketTaker_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","WicketsTaken"]),ignore_index=True)

    
for i in range(0,10):
    row=[["Mumbai Indians",highestWicketTaker_mi[0][i],highestWicketTaker_mi[1][i]]]
    
    highestWicketTaker_stats = highestWicketTaker_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","WicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Rajasthan Royals",highestWicketTaker_rr[0][i],highestWicketTaker_rr[1][i]]]
    
    highestWicketTaker_stats = highestWicketTaker_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","WicketsTaken"]),ignore_index=True)
    
for i in range(0,10):
    row=[["Sunrisers Hyderabad",highestWicketTaker_srh[0][i],highestWicketTaker_srh[1][i]]]
    
    highestWicketTaker_stats = highestWicketTaker_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","WicketsTaken"]),ignore_index=True)

for i in range(0,10):
    row=[["Royal Challengers Bangalore",highestWicketTaker_rcb[0][i],highestWicketTaker_rcb[1][i]]]
    
    highestWicketTaker_stats = highestWicketTaker_stats.append(pd.DataFrame(row, 
               columns=["TeamName","Player","WicketsTaken"]),ignore_index=True)
# highestWicketTaker_stats.head(20)

project.save_data(file_name='highestWicketTaker_stats.csv',data=highestWicketTaker_stats.to_csv())

{'file_name': 'highestWicketTaker_stats.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'ibmhc2021-donotdelete-pr-b4rupmtzwrnsav',
 'asset_id': 'ed32bf59-b6f6-47c5-a4bb-83b3ae8eeab8'}